In [1]:
import numpy as np
import pandas as pd
from talib import abstract

In [2]:
data = pd.read_csv('/Users/chungyu/畢業/0056.TW.csv')
data.set_index(pd.to_datetime(data['Date'],format='%Y/%m/%d'),inplace=True)

In [3]:
from talib import abstract
def data_process(data):

#股價漲跌幅
    DeltaClose = data["Close"].diff(1)
    data["股價漲跌幅"] = DeltaClose / data["Close"]

#布林通道帶寬 (均線週期、上軌標準差倍數、下軌標準差倍數、均線種類 tip31、tip40) 0 => SMA
    UpperBand,MiddleBand,DownBand = abstract.BBANDS(data["Close"], timeperiod=5, nbdevup=2.0, nbdevdn=2.0, matype=0)
    data["布林通道帶寬"] = (UpperBand-DownBand) / MiddleBand


#ＳＭＡ
    data["SMA5"] = abstract.SMA(data["Close"],timeperiod = 5)
    data["SMA10"] = abstract.SMA(data["Close"],timeperiod = 10)
    data["SMA20"] = abstract.SMA(data["Close"],timeperiod = 20)
    data["SMA60"] = abstract.SMA(data["Close"],timeperiod = 60)

#ＥＭＡ
    data["EMA5"] = abstract.EMA(data["Close"],timeperiod = 5)
    data["EMA10"] = abstract.EMA(data["Close"],timeperiod = 10)
    data["EMA20"] = abstract.EMA(data["Close"],timeperiod = 20)
    data["EMA60"] = abstract.EMA(data["Close"],timeperiod = 60)

#ＫＤ指標（ＳＴＯＣＨ）（週期、Ｋ值分母、Ｄ值分母）
    data["K"],data["D"] = abstract.STOCH(data["High"],data["Low"],data["Close"],fastk_period = 9, slowk_period = 3, slowd_period = 3)

#RSI
    data["RSI5"] = abstract.RSI(data["Close"],timeperiod = 5)
    data["RSI10"] = abstract.RSI(data["Close"],timeperiod = 10)

#MACD （短均線週期、長均線週期、ＭＡＣＤ週期）OSC＝DIF-MACD
    DIF,MACD,OSC = abstract.MACD(data["Close"],fastperiod = 12, slowperiod = 26, signalperiod = 9)
    data["MACD"] = OSC

#威廉指標
    data["WILLAR5"] = abstract.WILLR(data["High"],data["Low"],data["Close"],timeperiod = 5)
    data["WILLAR10"] = abstract.WILLR(data["High"],data["Low"],data["Close"],timeperiod = 10)
    
    return data

In [4]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
def split_data(start_time,end_time,k):
    X = data.loc[start_time:end_time,['布林通道帶寬','SMA5','SMA10','SMA20','SMA60','EMA5',
                                      'EMA10','EMA20','EMA60','K','D','RSI5','RSI10','MACD','WILLAR5','WILLAR10']]

    y_data = data.loc[start_time : end_time,['股價漲跌幅']]
    Y = np.where(y_data.shift(-1)> 0, 0 , 1 ) 
    
    #特徵篩選（ANOVA）
    k_best = SelectKBest(score_func=f_classif ,k = k)
    X_new = k_best.fit_transform(X,np.ravel(Y))

    selected_features = k_best.get_support(indices=True)
    print(selected_features)

    X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size =0.2, random_state = 0 )
    return X_train,X_test, y_train, y_test


In [5]:
from sklearn.svm import SVC
def SupportVector(X_train, X_test, y_train, y_test):
    clf = SVC()
    model = clf.fit(X_train,np.ravel(y_train))
    
    y_pre = clf.predict(X_test)
    
    from sklearn.metrics import classification_report
    from sklearn.metrics import classification_report, confusion_matrix
    
    print(confusion_matrix(y_test, y_pre))
    report = classification_report(y_test,y_pre)
    print(report)
    
    
    from sklearn import metrics
    # 績效
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pre)
    auc = metrics.auc(fpr, tpr)
    print('auc = ',auc)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test , y_pre)
    print("accuracy:",accuracy)
    
   
    
    return y_test, y_pre


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
def findbestCgamma (C,gama):
    param_grid = ({ 'kernel' : ['rbf'],'C' : C, 'gamma' : gama },{ 'kernel' : ['sigmoid'],'C' : C, 'gamma' : gama }, 
                 { 'kernel' : ['linear'],'C' : C, 'gamma' : gama })
    
    grid = GridSearchCV(SVC(),param_grid,n_jobs=-1,verbose=3)
    
    
    grid.fit(X_train,np.ravel(y_train))
    
    print('最適參數：', grid.best_params_)
    
    grid_predictions = grid.predict(X_test)
    
    print(confusion_matrix(y_test,grid_predictions))
    print('\n')
    print(classification_report(y_test,grid_predictions))
    
    from sklearn import metrics
    # 績效
    fpr, tpr, thresholds = metrics.roc_curve(y_test, grid_predictions)
    auc = metrics.auc(fpr, tpr)
    print('auc = ',auc)
    
    from sklearn.metrics import accuracy_score
    accuracy = accuracy_score(y_test , grid_predictions)
    print("accuracy:",accuracy)
    
    return y_test, grid_predictions

    
    

In [7]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2008-06-01','2008-12-31',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test )

import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)

[ 0  9 10 11 13]
[[ 1 12]
 [ 3 14]]
              precision    recall  f1-score   support

           0       0.25      0.08      0.12        13
           1       0.54      0.82      0.65        17

    accuracy                           0.50        30
   macro avg       0.39      0.45      0.38        30
weighted avg       0.41      0.50      0.42        30

auc =  0.45022624434389136
accuracy: 0.5


[I 2023-03-07 19:59:38,966] A new study created in memory with name: no-name-3d6b5e17-24ad-4a40-87c2-0283deb83f91
[I 2023-03-07 19:59:38,971] Trial 0 finished with value: 0.5666666666666667 and parameters: {'C': 53433.4136098886, 'gamma': 67082.95240803006, 'kernel': 'rbf'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:00:37,691] Trial 1 finished with value: 0.36666666666666664 and parameters: {'C': 98819.02204494951, 'gamma': 44093.69042346056, 'kernel': 'linear'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:01:02,777] Trial 2 finished with value: 0.4 and parameters: {'C': 40170.626407782416, 'gamma': 46844.836459739105, 'kernel': 'linear'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:01:02,780] Trial 3 finished with value: 0.5666666666666667 and parameters: {'C': 38611.51079655446, 'gamma': 23983.481873130575, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:01:48,114] Trial 4 finished

[I 2023-03-07 20:04:36,482] Trial 36 finished with value: 0.5666666666666667 and parameters: {'C': 40033.59927588008, 'gamma': 47997.78538068629, 'kernel': 'rbf'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:05:10,612] Trial 37 finished with value: 0.4 and parameters: {'C': 55797.35135792105, 'gamma': 82789.2557534028, 'kernel': 'linear'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:05:10,625] Trial 38 finished with value: 0.5666666666666667 and parameters: {'C': 34618.782122722914, 'gamma': 65513.70465232063, 'kernel': 'rbf'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:05:10,636] Trial 39 finished with value: 0.5666666666666667 and parameters: {'C': 51316.01733580652, 'gamma': 72039.04946488174, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:05:10,649] Trial 40 finished with value: 0.5666666666666667 and parameters: {'C': 27627.334047647746, 'gamma': 94452.61474165125, 'kernel': 'rb

[I 2023-03-07 20:11:02,973] Trial 73 finished with value: 0.5666666666666667 and parameters: {'C': 35594.42818483326, 'gamma': 589.2080785773596, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:11:02,987] Trial 74 finished with value: 0.5666666666666667 and parameters: {'C': 33915.74023448911, 'gamma': 12665.7896888544, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:11:02,999] Trial 75 finished with value: 0.5666666666666667 and parameters: {'C': 48327.951472245, 'gamma': 3827.9575179790204, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:11:37,937] Trial 76 finished with value: 0.36666666666666664 and parameters: {'C': 40993.542487004175, 'gamma': 13877.097435797656, 'kernel': 'linear'}. Best is trial 0 with value: 0.5666666666666667.
[I 2023-03-07 20:11:37,951] Trial 77 finished with value: 0.5666666666666667 and parameters: {'C': 32056.79289949018, 'gamma': 8657.4131

準確度： 0.5666666666666667
最佳參數 {'C': 53433.4136098886, 'gamma': 67082.95240803006, 'kernel': 'rbf'}


In [8]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2009-01-01','2010-01-31',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test )

import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)
#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

[I 2023-03-07 22:07:44,186] A new study created in memory with name: no-name-ad2a5cb0-b5a5-4ec6-9c34-19a3590590a3
[I 2023-03-07 22:07:44,191] Trial 0 finished with value: 0.6181818181818182 and parameters: {'C': 50981.43146780839, 'gamma': 72881.58848975706, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:07:44,198] Trial 1 finished with value: 0.6 and parameters: {'C': 70151.37466313991, 'gamma': 1320.050740683032, 'kernel': 'rbf'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:07:44,205] Trial 2 finished with value: 0.6181818181818182 and parameters: {'C': 19967.452396171047, 'gamma': 26027.598854855227, 'kernel': 'rbf'}. Best is trial 0 with value: 0.6181818181818182.


[1 2 5 6 7]
[[28  6]
 [13  8]]
              precision    recall  f1-score   support

           0       0.68      0.82      0.75        34
           1       0.57      0.38      0.46        21

    accuracy                           0.65        55
   macro avg       0.63      0.60      0.60        55
weighted avg       0.64      0.65      0.64        55

auc =  0.6022408963585434
accuracy: 0.6545454545454545


[I 2023-03-07 22:10:00,835] Trial 3 finished with value: 0.5636363636363636 and parameters: {'C': 44891.27104599721, 'gamma': 68459.84249738614, 'kernel': 'linear'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:10:04,916] Trial 4 finished with value: 0.4909090909090909 and parameters: {'C': 93899.03772755191, 'gamma': 73038.70752675939, 'kernel': 'linear'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:10:07,342] Trial 5 finished with value: 0.5454545454545454 and parameters: {'C': 55643.03190864042, 'gamma': 92994.58270392551, 'kernel': 'linear'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:10:07,347] Trial 6 finished with value: 0.6181818181818182 and parameters: {'C': 11773.502438833983, 'gamma': 32536.83131749988, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.6181818181818182.
[I 2023-03-07 22:10:07,354] Trial 7 finished with value: 0.6181818181818182 and parameters: {'C': 86113.97619689231, 'gamma': 32507.9784860537

[I 2023-03-07 22:11:31,449] Trial 40 finished with value: 0.6181818181818182 and parameters: {'C': 72228.77876674599, 'gamma': 70881.86396158092, 'kernel': 'sigmoid'}. Best is trial 8 with value: 0.6545454545454545.
[I 2023-03-07 22:11:31,471] Trial 41 finished with value: 0.6181818181818182 and parameters: {'C': 31046.19374620352, 'gamma': 28235.74071277872, 'kernel': 'rbf'}. Best is trial 8 with value: 0.6545454545454545.
[I 2023-03-07 22:11:31,492] Trial 42 finished with value: 0.6181818181818182 and parameters: {'C': 21462.423122345124, 'gamma': 5527.146360860206, 'kernel': 'rbf'}. Best is trial 8 with value: 0.6545454545454545.
[I 2023-03-07 22:11:31,512] Trial 43 finished with value: 0.6181818181818182 and parameters: {'C': 6752.832318293433, 'gamma': 22486.026813466357, 'kernel': 'rbf'}. Best is trial 8 with value: 0.6545454545454545.
[I 2023-03-07 22:11:31,534] Trial 44 finished with value: 0.6181818181818182 and parameters: {'C': 55184.66493702177, 'gamma': 14500.13928099562, 

[I 2023-03-07 22:13:14,898] Trial 76 finished with value: 0.6181818181818182 and parameters: {'C': 86906.52321288739, 'gamma': 12508.047517932715, 'kernel': 'sigmoid'}. Best is trial 51 with value: 0.7090909090909091.
[I 2023-03-07 22:13:22,053] Trial 77 finished with value: 0.5272727272727272 and parameters: {'C': 91454.09196775207, 'gamma': 2948.5612888081414, 'kernel': 'linear'}. Best is trial 51 with value: 0.7090909090909091.
[I 2023-03-07 22:13:22,071] Trial 78 finished with value: 0.5818181818181818 and parameters: {'C': 70582.88125840417, 'gamma': 174.58964001933327, 'kernel': 'rbf'}. Best is trial 51 with value: 0.7090909090909091.
[I 2023-03-07 22:13:25,834] Trial 79 finished with value: 0.5272727272727272 and parameters: {'C': 64418.822297067854, 'gamma': 27998.220713219212, 'kernel': 'linear'}. Best is trial 51 with value: 0.7090909090909091.
[I 2023-03-07 22:13:25,850] Trial 80 finished with value: 0.6181818181818182 and parameters: {'C': 82273.61169182083, 'gamma': 16641.

準確度： 0.7090909090909091
最佳參數 {'C': 59403.51984962192, 'gamma': 59819.01798352154, 'kernel': 'linear'}


In [10]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2011-08-01','2011-12-31',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test )
import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)

#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

[I 2023-03-07 22:25:20,759] A new study created in memory with name: no-name-eb8df8e3-9355-4576-9604-4cd2aacd934f
[I 2023-03-07 22:25:20,763] Trial 0 finished with value: 0.7727272727272727 and parameters: {'C': 55916.737289090146, 'gamma': 52705.49335871524, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:25:20,767] Trial 1 finished with value: 0.7727272727272727 and parameters: {'C': 14923.428350599099, 'gamma': 21627.66524397829, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.


[ 9 10 11 12 15]
[[ 2  3]
 [11  6]]
              precision    recall  f1-score   support

           0       0.15      0.40      0.22         5
           1       0.67      0.35      0.46        17

    accuracy                           0.36        22
   macro avg       0.41      0.38      0.34        22
weighted avg       0.55      0.36      0.41        22

auc =  0.37647058823529417
accuracy: 0.36363636363636365


[I 2023-03-07 22:25:32,288] Trial 2 finished with value: 0.5 and parameters: {'C': 16551.994846901674, 'gamma': 16160.856288917357, 'kernel': 'linear'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:25:56,773] Trial 3 finished with value: 0.5454545454545454 and parameters: {'C': 36458.598380005635, 'gamma': 34999.72393313019, 'kernel': 'linear'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:25:56,776] Trial 4 finished with value: 0.7727272727272727 and parameters: {'C': 56416.97048516022, 'gamma': 17548.24703710013, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:26:40,871] Trial 5 finished with value: 0.4090909090909091 and parameters: {'C': 11634.014057258268, 'gamma': 4024.1921817560033, 'kernel': 'linear'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:26:40,874] Trial 6 finished with value: 0.7727272727272727 and parameters: {'C': 20954.84482921236, 'gamma': 99383.50607455164, 'kernel': 'r

[I 2023-03-07 22:27:18,778] Trial 39 finished with value: 0.7727272727272727 and parameters: {'C': 9243.5003506346, 'gamma': 83962.88404645154, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:27:18,790] Trial 40 finished with value: 0.7727272727272727 and parameters: {'C': 29548.225891913615, 'gamma': 62547.03452299308, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:27:18,803] Trial 41 finished with value: 0.7727272727272727 and parameters: {'C': 80593.71187752578, 'gamma': 79129.48596822239, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:27:18,816] Trial 42 finished with value: 0.7727272727272727 and parameters: {'C': 99859.8717495804, 'gamma': 91739.50142685049, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:27:18,831] Trial 43 finished with value: 0.7727272727272727 and parameters: {'C': 2564.993482507547, 'gamma': 74703.03721825918, 'ker

[I 2023-03-07 22:28:13,436] Trial 76 finished with value: 0.7727272727272727 and parameters: {'C': 36622.755593983326, 'gamma': 78075.03075910578, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:28:13,451] Trial 77 finished with value: 0.7727272727272727 and parameters: {'C': 87714.59759985174, 'gamma': 93244.51636669275, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:28:13,467] Trial 78 finished with value: 0.7727272727272727 and parameters: {'C': 68828.11244743202, 'gamma': 62968.700569001856, 'kernel': 'rbf'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:28:13,482] Trial 79 finished with value: 0.7727272727272727 and parameters: {'C': 63375.37524389672, 'gamma': 32198.970381212926, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.7727272727272727.
[I 2023-03-07 22:28:13,497] Trial 80 finished with value: 0.7727272727272727 and parameters: {'C': 59687.908842586694, 'gamma': 85152.9930732

準確度： 0.7727272727272727
最佳參數 {'C': 55916.737289090146, 'gamma': 52705.49335871524, 'kernel': 'sigmoid'}


In [11]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2015-04-01','2015-08-31',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test)

import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)
#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023

[ 0  9 10 12 13]
[[ 0  9]
 [ 0 12]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         9
           1       0.57      1.00      0.73        12

    accuracy                           0.57        21
   macro avg       0.29      0.50      0.36        21
weighted avg       0.33      0.57      0.42        21

auc =  0.5
accuracy: 0.5714285714285714


[I 2023-03-07 23:12:48,632] Trial 5 finished with value: 0.5238095238095238 and parameters: {'C': 89459.46190604572, 'gamma': 16966.701726870735, 'kernel': 'linear'}. Best is trial 0 with value: 0.5714285714285714.
[I 2023-03-07 23:12:48,634] Trial 6 finished with value: 0.5714285714285714 and parameters: {'C': 49837.275192030706, 'gamma': 75185.08631913073, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5714285714285714.
[I 2023-03-07 23:12:48,636] Trial 7 finished with value: 0.5714285714285714 and parameters: {'C': 69769.9258496105, 'gamma': 19561.147556315496, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5714285714285714.
[I 2023-03-07 23:12:48,638] Trial 8 finished with value: 0.5714285714285714 and parameters: {'C': 38253.047223135836, 'gamma': 51065.957587017154, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.5714285714285714.
[I 2023-03-07 23:12:48,641] Trial 9 finished with value: 0.5714285714285714 and parameters: {'C': 8265.611495751256, 'gamma': 97728.13541

[I 2023-03-07 23:24:03,639] Trial 41 finished with value: 0.5238095238095238 and parameters: {'C': 70224.38316371154, 'gamma': 60345.360180001895, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:24:27,859] Trial 42 finished with value: 0.6190476190476191 and parameters: {'C': 76431.80690366401, 'gamma': 67222.3252910807, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:24:49,179] Trial 43 finished with value: 0.5238095238095238 and parameters: {'C': 81575.3547556267, 'gamma': 57473.10327076402, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:25:20,106] Trial 44 finished with value: 0.5238095238095238 and parameters: {'C': 70143.29860867676, 'gamma': 50455.13105543127, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:26:01,962] Trial 45 finished with value: 0.6190476190476191 and parameters: {'C': 61774.9799509405, 'gamma': 40693.78533

[I 2023-03-07 23:39:50,047] Trial 77 finished with value: 0.6190476190476191 and parameters: {'C': 79692.22316579992, 'gamma': 65929.88449298573, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:40:12,705] Trial 78 finished with value: 0.5238095238095238 and parameters: {'C': 89596.0794104298, 'gamma': 62745.31665787882, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:40:12,722] Trial 79 finished with value: 0.5714285714285714 and parameters: {'C': 68872.30378901443, 'gamma': 59725.37532448571, 'kernel': 'sigmoid'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:40:35,401] Trial 80 finished with value: 0.6190476190476191 and parameters: {'C': 82969.19609246425, 'gamma': 50522.186085038855, 'kernel': 'linear'}. Best is trial 18 with value: 0.6190476190476191.
[I 2023-03-07 23:41:21,855] Trial 81 finished with value: 0.5238095238095238 and parameters: {'C': 77564.68786920712, 'gamma': 66576.90

準確度： 0.6190476190476191
最佳參數 {'C': 98647.34205379624, 'gamma': 58368.61056440179, 'kernel': 'linear'}


In [12]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2018-10-01','2019-01-14',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test )

import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)
#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

[I 2023-03-07 23:49:57,455] A new study created in memory with name: no-name-01e277fd-41cd-45d2-b247-cc58e3dc7521
[I 2023-03-07 23:49:57,457] Trial 0 finished with value: 0.4 and parameters: {'C': 20363.817040081343, 'gamma': 13658.983617244901, 'kernel': 'rbf'}. Best is trial 0 with value: 0.4.


[ 1  5  6  9 10]
[[4 5]
 [2 4]]
              precision    recall  f1-score   support

           0       0.67      0.44      0.53         9
           1       0.44      0.67      0.53         6

    accuracy                           0.53        15
   macro avg       0.56      0.56      0.53        15
weighted avg       0.58      0.53      0.53        15

auc =  0.5555555555555555
accuracy: 0.5333333333333333


[I 2023-03-07 23:50:04,528] Trial 1 finished with value: 0.5333333333333333 and parameters: {'C': 85841.90791827033, 'gamma': 63544.843317471874, 'kernel': 'linear'}. Best is trial 1 with value: 0.5333333333333333.
[I 2023-03-07 23:50:11,566] Trial 2 finished with value: 0.5333333333333333 and parameters: {'C': 63431.02010724005, 'gamma': 62945.630164795264, 'kernel': 'linear'}. Best is trial 1 with value: 0.5333333333333333.
[I 2023-03-07 23:50:18,599] Trial 3 finished with value: 0.5333333333333333 and parameters: {'C': 70643.67033632282, 'gamma': 47228.58034804704, 'kernel': 'linear'}. Best is trial 1 with value: 0.5333333333333333.
[I 2023-03-07 23:50:18,602] Trial 4 finished with value: 0.4 and parameters: {'C': 94149.23568693757, 'gamma': 11584.349712538833, 'kernel': 'rbf'}. Best is trial 1 with value: 0.5333333333333333.
[I 2023-03-07 23:50:24,972] Trial 5 finished with value: 0.4 and parameters: {'C': 25780.314707465855, 'gamma': 88434.67554863478, 'kernel': 'linear'}. Best is

[I 2023-03-07 23:54:01,407] Trial 40 finished with value: 0.4 and parameters: {'C': 91508.75446961299, 'gamma': 67181.98611736172, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:54:11,074] Trial 41 finished with value: 0.4 and parameters: {'C': 28135.842115289415, 'gamma': 7087.550646216281, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:54:18,222] Trial 42 finished with value: 0.5333333333333333 and parameters: {'C': 81349.60539286076, 'gamma': 1876.0317108388626, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:54:27,826] Trial 43 finished with value: 0.4 and parameters: {'C': 12821.610647302234, 'gamma': 19976.864814211967, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:54:34,885] Trial 44 finished with value: 0.5333333333333333 and parameters: {'C': 47269.55419701045, 'gamma': 18132.859464161953, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:54:41,601] Trial 4

[I 2023-03-07 23:58:17,684] Trial 80 finished with value: 0.4 and parameters: {'C': 70686.77291090116, 'gamma': 44413.41542849655, 'kernel': 'sigmoid'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:58:25,577] Trial 81 finished with value: 0.4 and parameters: {'C': 14530.2534883999, 'gamma': 58347.85061439252, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:58:32,492] Trial 82 finished with value: 0.5333333333333333 and parameters: {'C': 55926.206993919826, 'gamma': 64058.884552306234, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:58:39,329] Trial 83 finished with value: 0.5333333333333333 and parameters: {'C': 48500.27145110484, 'gamma': 70691.64669951193, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:58:45,914] Trial 84 finished with value: 0.5333333333333333 and parameters: {'C': 44197.63964922734, 'gamma': 54376.17383949457, 'kernel': 'linear'}. Best is trial 14 with value: 0.6.
[I 2023-03-07 23:59:07,

準確度： 0.6
最佳參數 {'C': 2655.839374843672, 'gamma': 56596.59164473708, 'kernel': 'linear'}


In [13]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2020-01-01','2020-03-19',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test)


import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)
#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023

[ 3  6  7 14 15]
[[0 5]
 [0 5]]
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.50      1.00      0.67         5

    accuracy                           0.50        10
   macro avg       0.25      0.50      0.33        10
weighted avg       0.25      0.50      0.33        10

auc =  0.5
accuracy: 0.5


[I 2023-03-08 00:00:52,030] Trial 2 finished with value: 0.6 and parameters: {'C': 67870.28032189686, 'gamma': 52512.77678842711, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:00:52,032] Trial 3 finished with value: 0.5 and parameters: {'C': 23550.38166615935, 'gamma': 24807.607459523097, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:00:52,034] Trial 4 finished with value: 0.5 and parameters: {'C': 1980.9067317631545, 'gamma': 85945.76708007639, 'kernel': 'rbf'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:00:52,036] Trial 5 finished with value: 0.5 and parameters: {'C': 12510.805744751007, 'gamma': 23817.92027331291, 'kernel': 'rbf'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:00:52,038] Trial 6 finished with value: 0.5 and parameters: {'C': 9059.858635189843, 'gamma': 1781.4831034470458, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:00:52,040] Trial 7 finished with value: 0.5 and parameters: {'C

[I 2023-03-08 00:09:26,581] Trial 44 finished with value: 0.6 and parameters: {'C': 84149.16319083006, 'gamma': 73651.7940194844, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:09:40,203] Trial 45 finished with value: 0.6 and parameters: {'C': 77157.59055492257, 'gamma': 81374.59574423531, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:09:44,531] Trial 46 finished with value: 0.6 and parameters: {'C': 97540.76773850557, 'gamma': 44970.66306827247, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:09:44,540] Trial 47 finished with value: 0.5 and parameters: {'C': 88214.14038981016, 'gamma': 54081.60618301722, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:09:58,677] Trial 48 finished with value: 0.6 and parameters: {'C': 78862.9508325059, 'gamma': 34258.02743839935, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:09:58,685] Trial 49 finished with value: 0.5 and parameters

[I 2023-03-08 00:19:06,384] Trial 86 finished with value: 0.6 and parameters: {'C': 76994.7525435629, 'gamma': 59924.26403480829, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:19:20,197] Trial 87 finished with value: 0.6 and parameters: {'C': 62387.17945419291, 'gamma': 72998.31874348782, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:19:36,617] Trial 88 finished with value: 0.6 and parameters: {'C': 83866.13311511211, 'gamma': 42801.87239835648, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:19:48,806] Trial 89 finished with value: 0.6 and parameters: {'C': 64948.7782356464, 'gamma': 54167.58230419457, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:20:19,210] Trial 90 finished with value: 0.6 and parameters: {'C': 93144.65244952416, 'gamma': 48285.39563813194, 'kernel': 'linear'}. Best is trial 2 with value: 0.6.
[I 2023-03-08 00:20:38,947] Trial 91 finished with value: 0.6 and parameters:

準確度： 0.6
最佳參數 {'C': 67870.28032189686, 'gamma': 52512.77678842711, 'kernel': 'linear'}


In [14]:
data = data_process(data)

X_train, X_test, y_train, y_test = split_data('2020-03-20','2021-12-31',5)


y_test, y_pre = SupportVector(X_train,X_test, y_train, y_test)


import optuna
def objective(trial):
    
    c = trial.suggest_float('C',0,1e+5)
    gamma = trial.suggest_float('gamma',0,1e+5)
    kernel = trial.suggest_categorical('kernel',['rbf','linear','sigmoid'])
    
    svc_new = SVC(C = c, gamma = gamma , kernel = kernel)
    svc_new.fit(X_train ,np.ravel(y_train))
    from sklearn import metrics
    from sklearn.metrics import accuracy_score
    y_pre_new = svc_new.predict(X_test)
    auc_optuna = accuracy_score(y_test ,y_pre_new )
    
    return(auc_optuna)

study = optuna.create_study(direction= "maximize")
study.optimize(objective, n_trials=100)

best_trial = study.best_trial
best_accuracy = best_trial.value

best_params = study.best_params
print('準確度：',best_accuracy)
print('最佳參數',best_params)
#y_test,grid_predictions = findbestCgamma ([0.01,0.1,1,10,100,1000],[0.01,0.1,1,10,100,1000])

/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/tensorflow/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2023

[1 2 5 6 7]
[[45  0]
 [44  0]]
              precision    recall  f1-score   support

           0       0.51      1.00      0.67        45
           1       0.00      0.00      0.00        44

    accuracy                           0.51        89
   macro avg       0.25      0.50      0.34        89
weighted avg       0.26      0.51      0.34        89

auc =  0.5
accuracy: 0.5056179775280899


[I 2023-03-08 00:22:32,193] Trial 2 finished with value: 0.5168539325842697 and parameters: {'C': 58105.69406889095, 'gamma': 30514.617968913393, 'kernel': 'linear'}. Best is trial 2 with value: 0.5168539325842697.
[I 2023-03-08 00:22:32,204] Trial 3 finished with value: 0.5056179775280899 and parameters: {'C': 5437.343108039195, 'gamma': 59494.936320331726, 'kernel': 'rbf'}. Best is trial 2 with value: 0.5168539325842697.
[I 2023-03-08 00:22:32,215] Trial 4 finished with value: 0.5056179775280899 and parameters: {'C': 19907.85699286379, 'gamma': 16773.745599271882, 'kernel': 'rbf'}. Best is trial 2 with value: 0.5168539325842697.
[I 2023-03-08 00:22:32,221] Trial 5 finished with value: 0.5056179775280899 and parameters: {'C': 51288.67333809408, 'gamma': 61178.81511478882, 'kernel': 'sigmoid'}. Best is trial 2 with value: 0.5168539325842697.
[I 2023-03-08 00:22:33,082] Trial 6 finished with value: 0.6067415730337079 and parameters: {'C': 98283.69401769519, 'gamma': 34713.96150119994, '

[I 2023-03-08 00:23:10,470] Trial 39 finished with value: 0.5280898876404494 and parameters: {'C': 78973.78213909602, 'gamma': 23788.811683876353, 'kernel': 'linear'}. Best is trial 6 with value: 0.6067415730337079.
[I 2023-03-08 00:23:10,486] Trial 40 finished with value: 0.5056179775280899 and parameters: {'C': 93225.26064343158, 'gamma': 44693.777311851605, 'kernel': 'rbf'}. Best is trial 6 with value: 0.6067415730337079.
[I 2023-03-08 00:23:12,830] Trial 41 finished with value: 0.48314606741573035 and parameters: {'C': 77154.74415681334, 'gamma': 95056.45750879368, 'kernel': 'linear'}. Best is trial 6 with value: 0.6067415730337079.
[I 2023-03-08 00:23:13,377] Trial 42 finished with value: 0.5280898876404494 and parameters: {'C': 84178.01209758145, 'gamma': 93622.57810460177, 'kernel': 'linear'}. Best is trial 6 with value: 0.6067415730337079.
[I 2023-03-08 00:23:14,248] Trial 43 finished with value: 0.4943820224719101 and parameters: {'C': 80739.86378302408, 'gamma': 89227.8320646

[I 2023-03-08 00:23:47,018] Trial 75 finished with value: 0.4943820224719101 and parameters: {'C': 78383.22821722753, 'gamma': 99989.0419978163, 'kernel': 'linear'}. Best is trial 52 with value: 0.6179775280898876.
[I 2023-03-08 00:23:47,702] Trial 76 finished with value: 0.6179775280898876 and parameters: {'C': 85353.75051490532, 'gamma': 87249.63038191311, 'kernel': 'linear'}. Best is trial 52 with value: 0.6179775280898876.
[I 2023-03-08 00:23:48,715] Trial 77 finished with value: 0.6292134831460674 and parameters: {'C': 85616.10798205691, 'gamma': 88538.23251869969, 'kernel': 'linear'}. Best is trial 77 with value: 0.6292134831460674.
[I 2023-03-08 00:23:49,840] Trial 78 finished with value: 0.5730337078651685 and parameters: {'C': 85100.66588885685, 'gamma': 86402.01735289389, 'kernel': 'linear'}. Best is trial 77 with value: 0.6292134831460674.
[I 2023-03-08 00:23:51,378] Trial 79 finished with value: 0.5056179775280899 and parameters: {'C': 92142.69022677126, 'gamma': 79283.9803

準確度： 0.6292134831460674
最佳參數 {'C': 85616.10798205691, 'gamma': 88538.23251869969, 'kernel': 'linear'}
